In [1]:
# !pip3 install malaya -U --no-deps

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import tensorflow as tf
import tensorflow_text  

sess = tf.InteractiveSession()
meta_graph_def = tf.compat.v1.saved_model.load(sess, ["serve"], 'output/1603124082')
signature_def = meta_graph_def.signature_def["serving_default"]
pred = lambda x: sess.run(
    fetches=signature_def.outputs["outputs"].name, 
    feed_dict={signature_def.inputs["input"].name: x}
)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from output/1603124082/variables/variables


In [4]:
import malaya
malaya.__version__, malaya.__gpu__

('3.9.1', ['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3'])

In [5]:
model = malaya.true_case.transformer(model = 'small')

In [7]:
from glob import glob
import json

negatives = glob('negative/*.json')
negatives

['negative/clean-gossipcop_real2.json',
 'negative/clean-gossipcop_real3.json',
 'negative/1.json',
 'negative/clean-gossipcop_real1.json',
 'negative/clean-politifact_real.json',
 'negative/2.json']

In [8]:
texts = []
for n in negatives:
    with open(n) as fopen:
        texts.extend(json.load(fopen))
len(texts)

26832

In [9]:
%%time

s = model.true_case([texts[0]])[0]

CPU times: user 4.47 s, sys: 557 ms, total: 5.03 s
Wall time: 2.33 s


In [11]:
import re

# minimum cleaning, just simply to remove newlines.
def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

string = cleaning(s)
string

'Jadi dadah drak terbang ke Memphis untuk mengambil sepupunya Hyfr Jalaah untuk Prom. Fairleyhighschool Prom2k17 Family Unconditionallove drake Ovo Globalambassador Aubrih Welcomeovo Ovosounda post shared by Rhenna X Dhrek Papixriri pada 13 Mei 2017 at 9 02pm PDT.'

In [12]:
from tqdm import tqdm

r = []
for i in tqdm(range(len(texts))):
    try:
        splitted = texts[i].split()
        if len(splitted) >= 100:
            s = ' '.join(splitted[:800])
            s = model.true_case([s])[0]
            p = pred([f'ringkasan: {cleaning(s)}'] * 5).tolist()
            p = [i.decode() for i in p]
            r.append((s, p))
    except Exception as e:
        print(e)

 19%|█▉        | 5215/26832 [27:23:38<113:33:09, 18.91s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/husein/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-b89c1ff57356>", line 9, in <module>
    s = model.true_case([s])[0]
  File "/home/husein/.local/lib/python3.6/site-packages/herpetologist/__init__.py", line 88, in check
    return func(*args, **kwargs)
  File "/home/husein/.local/lib/python3.6/site-packages/malaya/model/tf.py", line 508, in true_case
    return self._true_case(strings, beam_search = beam_search)
  File "/home/husein/.local/lib/python3.6/site-packages/malaya/model/tf.py", line 488, in _true_case
    p = self._sess.run(output, feed_dict = {self._X: batch_x}).tolist()
  File "/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py", line 956, in run
    run_metadata_ptr)
  File "/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.


KeyboardInterrupt



In [18]:
json

<module 'json' from '/usr/lib/python3.6/json/__init__.py'>

In [19]:
with open('fake-news-negatives-summarization.json', 'w') as fopen:
    json.dump(r, fopen)